## DIA 044: Implementacion de un Sistema de Auditoria y Trazabilidad

En el Día 44, se implementará un sistema de auditoría y trazabilidad para la API. Este sistema registra las acciones críticas realizadas por los usuarios (como inicios de sesión, solicitudes de predicción, envíos de feedback y operaciones administrativas) en una tabla de auditoría. La trazabilidad es esencial para monitorear la seguridad, cumplir con requisitos regulatorios y diagnosticar problemas. Además, se añade un endpoint protegido para que los administradores puedan consultar estos registros de auditoría.

Código Completo (api.py)
python
Copiar
import os
import io
import random
import json
import time
import threading
import logging
from datetime import datetime
from functools import wraps

from flask import Flask, request, jsonify, render_template, url_for
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_bcrypt import Bcrypt
from flask_mail import Mail, Message
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
from flask_socketio import SocketIO, emit, join_room
from flasgger import Swagger

# Configuración básica y variables de entorno
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'your_secret_key')
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'your_jwt_secret_key')
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///app.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Configuración de Flasgger para documentación
swagger_config = {
    "headers": [],
    "specs": [
        {
            "endpoint": "apispec_1",
            "route": "/apispec_1.json",
            "rule_filter": lambda rule: True,  
            "model_filter": lambda tag: True,  
        }
    ],
    "static_url_path": "/flasgger_static",
    "swagger_ui": True,
    "specs_route": "/docs/"
}
swagger = Swagger(app, config=swagger_config)

# Inicialización de extensiones
db = SQLAlchemy(app)
migrate = Migrate(app, db)
bcrypt = Bcrypt(app)
mail = Mail(app)
jwt = JWTManager(app)
limiter = Limiter(app, key_func=get_remote_address, default_limits=["200 per day", "50 per hour"])
socketio = SocketIO(app, cors_allowed_origins="*")

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------
# Modelos
# ---------------------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    # Se pueden añadir otros campos según necesidad (email, role, etc.)

class Feedback(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    prediction = db.Column(db.Integer, nullable=False)
    correct = db.Column(db.Boolean, nullable=False)
    comment = db.Column(db.Text, nullable=True)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

    def to_dict(self):
        return {
            "id": self.id,
            "username": self.username,
            "prediction": self.prediction,
            "correct": self.correct,
            "comment": self.comment,
            "timestamp": self.timestamp.isoformat()
        }

# Nuevo Modelo: AuditLog para registrar eventos críticos
class AuditLog(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    action = db.Column(db.String(120), nullable=False)
    details = db.Column(db.Text, nullable=True)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

    def to_dict(self):
        return {
            "id": self.id,
            "username": self.username,
            "action": self.action,
            "details": self.details,
            "timestamp": self.timestamp.isoformat()
        }

# ---------------------------
# Función de Auditoría
# ---------------------------
def log_audit_event(username, action, details=""):
    audit = AuditLog(username=username, action=action, details=details)
    db.session.add(audit)
    db.session.commit()
    logger.info(f"Audit log: {username} - {action} - {details}")

# ---------------------------
# Decorador para Roles (simplificado)
# ---------------------------
def role_required(required_role):
    def decorator(f):
        @wraps(f)
        def wrapper(*args, **kwargs):
            current_user_identity = get_jwt_identity()
            if not current_user_identity:
                return jsonify({"msg": "Token de acceso requerido"}), 401
            user = User.query.filter_by(username=current_user_identity).first()
            if not user:
                return jsonify({"msg": "Usuario no encontrado"}), 404
            if getattr(user, 'role', 'user') != required_role:
                return jsonify({"msg": "Acceso no autorizado"}), 403
            return f(*args, **kwargs)
        return wrapper
    return decorator

# ---------------------------
# Endpoints Comunes
# ---------------------------
@app.route('/login', methods=['POST'])
def login():
    """
    User Login
    ---
    tags:
      - Auth
    parameters:
      - in: body
        name: credentials
        schema:
          type: object
          required:
            - username
            - password
          properties:
            username:
              type: string
            password:
              type: string
    responses:
      200:
        description: JWT token generado.
    """
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    # Se asume autenticación correcta para este ejemplo.
    access_token = create_access_token(identity=username)
    log_audit_event(username, "login", "Usuario inició sesión")
    logger.info(f"Usuario '{username}' inició sesión.")
    return jsonify(access_token=access_token), 200

@app.route('/health', methods=['GET'])
def health():
    """
    Health Check
    ---
    tags:
      - Health
    responses:
      200:
        description: La aplicación está funcionando correctamente.
    """
    return jsonify({"status": "ok"}), 200

# ---------------------------
# Endpoints de Versionado (v1 y v2) (para referencia)
# ---------------------------
api_v1 = Blueprint('api_v1', __name__)
@api_v1.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("100 per day")
def predict_v1():
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400
    result = {"prediccion": 5, "probabilidad": 0.90, "version": "v1"}
    logger.info("v1: Predicción realizada.")
    log_audit_event(get_jwt_identity(), "predict_v1", "Predicción v1 realizada")
    return jsonify(result), 200

api_v2 = Blueprint('api_v2', __name__)
@api_v2.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("150 per day")
def predict_v2():
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400
    result = {"prediccion": 7, "probabilidad": 0.95, "version": "v2", "mensaje": "Predicción mejorada"}
    logger.info("v2: Predicción realizada con mejoras.")
    log_audit_event(get_jwt_identity(), "predict_v2", "Predicción v2 realizada")
    return jsonify(result), 200

app.register_blueprint(api_v1, url_prefix='/api/v1')
app.register_blueprint(api_v2, url_prefix='/api/v2')

# ---------------------------
# Endpoint de A/B Testing para la Predicción (para referencia)
# ---------------------------
ab_blueprint = Blueprint('api_ab', __name__)
@ab_blueprint.route('/predict', methods=['POST'])
@jwt_required()
def predict_ab():
    version = random.choice(['v1', 'v2'])
    if version == 'v1':
        response = predict_v1()
    else:
        response = predict_v2()
    data = response.get_json()
    data['ab_version'] = version
    logger.info(f"A/B Testing: Solicitud enrutada a la versión {version}.")
    log_audit_event(get_jwt_identity(), "predict_ab", f"Solicitud enrutada a {version}")
    return jsonify(data), response.status_code

app.register_blueprint(ab_blueprint, url_prefix='/api/ab')

# ---------------------------
# Endpoint de Retraining con MLFlow (para referencia)
# ---------------------------
@app.route('/admin/retrain_mlflow', methods=['POST'])
@jwt_required()
@role_required('admin')
def retrain_mlflow():
    mlflow.set_experiment("Retraining Experiment")
    with mlflow.start_run() as run:
        logger.info("Inicio del retraining con MLFlow...")
        time.sleep(10)  # Simula retraining
        mlflow.log_param("learning_rate", 0.001)
        mlflow.log_metric("accuracy", 0.92)
        artifact_path = "model_info.txt"
        with open(artifact_path, "w") as f:
            f.write("Modelo actualizado basado en feedback con MLFlow.")
        mlflow.log_artifact(artifact_path)
        logger.info("Retraining completado y registrado en MLFlow.")
        log_audit_event(get_jwt_identity(), "retrain_mlflow", f"Run ID: {run.info.run_id}")
        return jsonify({"msg": "Retraining experiment logged in MLFlow", "run_id": run.info.run_id}), 202

# ---------------------------
# Endpoint de Slack Alert (para referencia)
# ---------------------------
def send_slack_notification(message):
    webhook_url = os.getenv("SLACK_WEBHOOK_URL")
    if not webhook_url:
        logger.error("SLACK_WEBHOOK_URL no está configurado.")
        return False
    payload = {"text": message}
    headers = {"Content-Type": "application/json"}
    response = requests.post(webhook_url, data=json.dumps(payload), headers=headers)
    if response.status_code != 200:
        logger.error("Error al enviar notificación a Slack: %s", response.text)
        return False
    return True

@app.route('/admin/slack_alert', methods=['POST'])
@jwt_required()
@role_required('admin')
def slack_alert():
    data = request.get_json()
    message = data.get("message", "Alerta de la API")
    if send_slack_notification(message):
        logger.info("Notificación Slack enviada: " + message)
        log_audit_event(get_jwt_identity(), "slack_alert", message)
        return jsonify({"msg": "Slack alert sent successfully"}), 200
    else:
        return jsonify({"msg": "Failed to send Slack alert"}), 500

# ---------------------------
# Nuevo Endpoint: Sistema de Auditoría y Trazabilidad
# ---------------------------
@app.route('/admin/audit_logs', methods=['GET'])
@jwt_required()
@role_required('admin')
def get_audit_logs():
    """
    Obtiene los registros de auditoría.
    ---
    tags:
      - Audit
    responses:
      200:
        description: Lista de registros de auditoría.
    """
    logs = AuditLog.query.order_by(AuditLog.timestamp.desc()).all()
    log_list = [log.to_dict() for log in logs]
    return jsonify(log_list), 200

# Modelo de Auditoría
class AuditLog(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    action = db.Column(db.String(120), nullable=False)
    details = db.Column(db.Text, nullable=True)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

    def to_dict(self):
        return {
            "id": self.id,
            "username": self.username,
            "action": self.action,
            "details": self.details,
            "timestamp": self.timestamp.isoformat()
        }

# ---------------------------
# Endpoint de Reporte Automatizado (para referencia)
# ---------------------------
@app.route('/admin/report', methods=['GET'])
@jwt_required()
@role_required('admin')
def generate_report():
    total_feedback = Feedback.query.count()
    if total_feedback == 0:
        return jsonify({"msg": "No hay feedback disponible"}), 200
    correct_feedback = Feedback.query.filter_by(correct=True).count()
    incorrect_feedback = Feedback.query.filter_by(correct=False).count()
    accuracy = (correct_feedback / total_feedback) * 100
    report = {
        "total_feedback": total_feedback,
        "correct_feedback": correct_feedback,
        "incorrect_feedback": incorrect_feedback,
        "accuracy_percentage": accuracy
    }
    logger.info("Reporte generado: " + json.dumps(report))
    log_audit_event(get_jwt_identity(), "generate_report", "Reporte generado")
    return jsonify(report), 200

# ---------------------------
# Ejecutar la aplicación
# ---------------------------
if __name__ == '__main__':
    socketio.run(app, debug=True)
Explicación del Código
Integración de MLFlow, Slack y Auditoría:

Se integró MLFlow para registrar experimentos de retraining en el endpoint /admin/retrain_mlflow.
Se implementó un endpoint /admin/slack_alert para enviar notificaciones a Slack mediante un webhook.
Se añadió un nuevo modelo AuditLog y un endpoint /admin/audit_logs para registrar y consultar los eventos críticos, permitiendo la trazabilidad de acciones en la API.
Registro de Eventos:

La función log_audit_event registra eventos importantes en la base de datos para futuras auditorías.
Los endpoints críticos (login, predicción, retraining, generación de reportes y alertas de Slack) llaman a esta función para guardar registros de auditoría.
Protección y Seguridad:

Los endpoints administrativos están protegidos mediante JWT y un decorador role_required, asegurando que solo los administradores puedan acceder a ellos.
Ejecución y Documentación:

La aplicación se ejecuta con soporte para WebSockets (Flask-SocketIO) y la documentación interactiva se genera mediante Flasgger.
